In [1]:
from sympy import *
init_printing()

In [2]:
def skew(l):
    l1, l2, l3 = l
    return Matrix([
        [0, -l3, l2],
        [l3, 0, -l1],
        [-l2, l1, 0]
    ])

In [3]:
# define state variables
x, y, z, eta0, eps1, eps2, eps3, u, v, w, p, q, r = symbols('x y z et0 eps1 eps2 eps3 u v w p q r', real=True)
s = Matrix([x, y, z, eta0, eps1, eps2, eps3, u, v, w, p, q, r])

# position and orientation
eta = Matrix([x, y, z, eta0, eps1, eps2, eps3])
nu = Matrix([u, v, w, p, q, r])

# centre of gravity
xg, yg, zg = symbols('xg yg zg', real=True)
rg = Matrix([xg, yg, zg])

# centre of bouyancy
xb, yb, zb = symbols('xb yb zb', real=True)
rb = Matrix([xb, yb, zb])

# center of pressure
xcp, ycp, zcp = symbols('xcp ycp zcp', real=True)
rcp = Matrix([xcp, ycp, zcp])

# mass matrix
m = symbols('m', real=True, positive=True)
Ixx, Iyy, Izz = symbols('Ixx Iyy Izz')
I0 = diag(Ixx, Iyy, Izz)
M = BlockMatrix([
    [m*eye(3), -m*skew(rg)],
    [m*skew(rg), I0]
])
M = Matrix(M)
# M = simplify(M)

# Coriolis and centripetal matrix
nu1 = Matrix([u, v, w])
nu2 = Matrix([p, q, r])
crb = BlockMatrix([
    [zeros(3), -m*skew(nu1)-m*skew(nu2)*skew(rg)],
    [-m*skew(nu1)+m*skew(rg)*skew(nu2), -skew(I0*nu2)]
])
crb = Matrix(crb)
# crb = simplify(crb)

# damping matrix
Xuu, Yvv, Zww, Kpp, Mqq, Nrr = symbols(
    'Xuu Yvv Zww Kpp Mqq Nrr', real=True
)
D = Matrix([
    [Xuu*abs(u), 0, 0, 0, 0, 0],
    [0, Yvv*abs(v), 0, 0, 0, 0],
    [0, 0, Zww*abs(w), 0, 0, 0],
    [0, -zcp*Yvv*abs(v), ycp*Zww*abs(w), Kpp*abs(p), 0, 0],
    [zcp*Xuu*abs(u), 0, -xcp*Zww*abs(w), 0, Mqq*abs(q), 0],
    [-ycp*Xuu*abs(u), xcp*Yvv*abs(v), 0, 0, 0, Nrr*abs(r)]
])
# D = simplify(D)

# rotational transform between body and NED quaternions
Tq = Rational(1,2)*Matrix([
    [-eps1, -eps2, -eps3],
    [eta0, -eps3, eps2],
    [eps3, eta0, -eps1],
    [-eps2, eps1, eta0]
])
# Tq = simplify(Tq)

Rq = Matrix([
    [1-2*(eps2**2+eps3**2), 2*(eps1*eps2-eps3*eta0), 2*(eps1*eps3+eps2*eta0)],
    [2*(eps1*eps2+eps3*eta0), 1-2*(eps1**2+eps3**2), 2*(eps2*eps3-eps1*eta0)],
    [2*(eps1*eps3-eps2*eta0), 2*(eps2*eps3+eps1*eta0), 1-2*(eps1**2+eps2**2)]
])
Jeta = BlockMatrix([
    [Rq, zeros(3)],
    [zeros(4,3), Tq]
])
Jeta = Matrix(Jeta)
# Jeta = simplify(Jeta)

# bouyancy in quaternions
W, B = symbols('W B', real=True)
fg = Matrix([0, 0, W])
fb = Matrix([0, 0, -B])
Rqinv = Rq.inv()
geta = Matrix([
    Rqinv*(fg+fb),
    skew(rg)*Rqinv*fg + skew(rb)*Rqinv*fb
])
# geta = simplify(geta)

In [7]:
print(cse(Jeta))

([(x0, -2*eps2**2), (x1, 1 - 2*eps3**2), (x2, 2*eps2), (x3, eps1*x2), (x4, 2*eps3), (x5, et0*x4), (x6, eps1*x4), (x7, et0*x2), (x8, -2*eps1**2), (x9, 2*eps1*et0), (x10, eps2*x4), (x11, eps1/2), (x12, -x11), (x13, eps2/2), (x14, -x13), (x15, eps3/2), (x16, -x15), (x17, et0/2)], [Matrix([
[x0 + x1,  x3 - x5,     x6 + x7,   0,   0,   0],
[x3 + x5,  x1 + x8,    x10 - x9,   0,   0,   0],
[x6 - x7, x10 + x9, x0 + x8 + 1,   0,   0,   0],
[      0,        0,           0, x12, x14, x16],
[      0,        0,           0, x17, x16, x13],
[      0,        0,           0, x15, x17, x12],
[      0,        0,           0, x14, x11, x17]])])


In [4]:
# thrust model
Kt0, Kt1 = symbols('Kt0 Kt1', real=True)
Kt = Matrix([Kt0, Kt1])
Qt0, Qt1 = symbols('Qt0 Qt1', real=True)
Qt = Matrix([Qt0, Qt1])

# control inputs
rpm0, rpm1 = symbols('rpm0 rpm1', real=True)
rpm = Matrix([rpm0, rpm1])
de, dr = symbols('de dr', real=True)
control_vector = Matrix([rpm0, rpm1, de, dr])

# control force vector
Ft = Kt.dot(rpm)
Mt = Qt.dot(rpm)

# coefficient for each element in cost function
tauc = Matrix([
    Ft*cos(de)*cos(dr),
    -Ft*sin(dr),
    Ft*sin(de)*cos(dr),
    Mt*cos(de)*cos(dr),
    -Mt*sin(dr),
    Mt*sin(de)*cos(dr)
])

In [5]:
etadot = Jeta*nu
nudot = M.inv()*(tauc - (crb + D)*nu - geta)

In [6]:
sdot = Matrix([
   etadot,
   nudot 
])

In [ ]:
print(list(set(sdot.free_symbols) - set(s.free_symbols) - set(control_vector.free_symbols)))

In [7]:
# Lagrangian
alpha = symbols('\\alpha', real=True, positive=True)
L = alpha + (1-alpha)*tauc.norm()

In [8]:
l = Matrix([symbols('lambda_{}'.format(var)) for var in s])

In [9]:
H = l.dot(sdot) + L

In [10]:
eq = H.diff(control_vector)

In [11]:
sol = solve(eq, control_vector)

KeyboardInterrupt: 

In [ ]:
L.diff(control_vector)

In [ ]:
tauc.transpose()*tauc